In [ ]:
pip install pyarrow


In [ ]:
pip install fastparquet

In [140]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime


Consigna: def userdata(User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items. Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [ ]:
''' Lectura de Archivos '''
# Se abren los 3 dataframes por que el data frame de la función necesita por lo menos un dato de cada uno

df_steam_games = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_steam_games.parquet"
df_steam_games = pd.read_parquet(df_steam_games )
df_steam_games

In [ ]:
# Cambio el nombre app_name por item_name para poder hacer merge con users_items

df_steam_games = df_steam_games.rename(columns={'app_name': 'item_name'})
df_steam_games.head()

In [ ]:
# Ubicación local archivo user_reviews https://github.com/MarceCorreal/PI1_MachineLearning/blob/main/Funciones/Developer/df_developer.parquet
df_user_reviews = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_user_reviews.parquet"
df_user_reviews = pd.read_parquet(df_user_reviews)
df_user_reviews

In [ ]:
df_users_items = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_users_items.parquet"
df_users_items = pd.read_parquet(df_users_items)
df_users_items 

In [ ]:
# Primero borro de steam Games todas las columnas menos item_name y price para poder hacer merge y que me quepa

df_steam_games.head()

In [146]:
columnas_borrar =['publisher',	'genres',	'url',	'release_date',	'reviews_url',	'specs',	'early_access',	'id',	'developer',	'release_year']

In [147]:
df_steam_games = df_steam_games.drop(columnas_borrar, axis=1)

In [ ]:
df_steam_games 

In [ ]:
df_users_items.head()

In [150]:
# borro columnas que no me sirven

columnas_borrar2 = ['user_url',	'playtime_forever',	'playtime_2weeks']

In [151]:

df_users_items = df_users_items.drop(columnas_borrar2,axis=1)


In [ ]:
df_users_items

In [ ]:
# Inicio haciendo un dataframe auxiliar1 con item:id y price para lo cual hago merge entre steam_games y user_id a travez de item_name

df2_aux_3 = pd.merge(df_steam_games, df_users_items, on='item_name')
df2_aux_3

In [154]:
# Convertir la columna 'price' a tipo de datos float
df2_aux_3['price'] = df2_aux_3['price'].astype(float)

In [ ]:
# Selecciona columnas para crear un nuevo DataFrame a partir de users_reviews con user reviews y reviews recommend

df_user_reviews.head()

In [156]:
Columnas_borrar3 =['user_url',	'posted',	'item_id',	'sentiment_analysis']

In [128]:
df_user_reviews = df_user_reviews.drop(Columnas_borrar3, axis=1)


In [ ]:
# Se genera la columna cantidad total gastado
df2_aux_3['cantidad total gastado'] = df2_aux_3['price'] * df2_aux_3['items_count']
df2_aux_3

In [162]:
# Ahora hadgo df con user_id y recommend basado en user_reviews

df_user_reviews.head()
 

,user_id,user_url,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted November 5, 2011.",1250,True,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted July 15, 2011.",22200,True,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"Posted April 21, 2011.",43110,True,2
3,js41637,http://steamcommunity.com/id/js41637,"Posted June 24, 2014.",251610,True,2
4,js41637,http://steamcommunity.com/id/js41637,"Posted September 8, 2013.",227300,True,0


In [163]:
columnas_borrar3 = ['user_url',	'posted',	'item_id',	'sentiment_analysis']

In [164]:
df_user_reviews = df_user_reviews.drop(columnas_borrar3, axis=1)

In [ ]:
df_user_reviews

In [168]:
df2_aux_4 = df_user_reviews

In [171]:
# Verifico  la columna reviews_recommend y vemos que tipo de valores tiene

valores_unicos = df2_aux_4['recommend'].unique()
valores_unicos

array([ True, False])

In [173]:
# Pasamos todos los valores de la columna a booleanos

df2_aux_4['recommend'] = df2_aux_4['recommend'].astype(bool)

In [175]:
# Veo unique

valores_unicos = df2_aux_4['recommend'].unique()


In [176]:
# Verifico  el tipo de dato de la columna.

valores_unicos.dtype


dtype('bool')

In [179]:
# Trabajo el df para ver los porcentajes

df2_aux_5 = pd.DataFrame(df2_aux_4)


In [181]:
# Creo columnas 'total_true' y 'total_false'
df2_aux_5['total_true'] = df2_aux_5['recommend'].astype(int)


In [182]:
df2_aux_5['total_false'] = (~df2_aux_5['recommend']).astype(int)

In [ ]:
df2_aux_5.head()

In [186]:
# Agrupar por 'user_id' y sumar las columnas correspondientes
df2_aux_6 = df2_aux_5.groupby('user_id').agg({'total_true': 'sum', 'total_false': 'sum'})

In [188]:
# Creo  columna 'total_recommend'
df2_aux_6['total_recommend'] = df2_aux_6['total_true'] + df2_aux_6['total_false']

In [189]:
# Creao la columna 'percentage_true' con  signo de porcentaje

df2_aux_6['percentage_true'] = (df2_aux_6['total_true'] / df2_aux_6['total_recommend']) * 100

In [191]:
df2_aux_7 = pd.merge(df2_aux_3, df2_aux_6, on='user_id', how='inner')

In [ ]:
df2_aux_7.head()

In [193]:
dataset_endpoint_2 = df2_aux_7

In [ ]:
dataset_endpoint_2.to_parquet('dataset_endpoint_2.parquet')
